# Import statements
Following packages are necessary:

In [4]:
import sys
sys.path.append("D:\Julian\workspace\endolas\packages")

import random
from endolas import utils
from endolas import ccall

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from endolas import UNet
from endolas import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from endolas import LASTENSequence
from matplotlib import pyplot as plt

# Checks
The version of tensorflow as well as the GPU support are checked.

In [5]:
print(tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

2.1.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 3.1) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN2 data set.

# Data
Import training images

In [18]:
tf.keras.backend.clear_session()

path_train = r'D:\Julian\data\LASTEN2\train'
path_validation = r'D:\Julian\data\LASTEN2\validation'
path_test = r'D:\Julian\data\LASTEN2\test'

#path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/train'
#path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/validation'
#path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/test'

width = 384 #768
height = 384 #768

# Seeding
Seeds are set to ensure reproducible training.

In [19]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [20]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 384, 384, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 384, 384, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 384, 384, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 384, 384, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [21]:
train_gen = LASTENSequence(path_train,
                           batch_size=1,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=1,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=2,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [22]:
aim_path= "3_1_segmentation"

save_path = "../results/" + aim_path

logger = CSVLogger(save_path + "/log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5", period=10)
checker_best = ModelCheckpoint(save_path + "/best_weights.hdf5", save_best_only=True)
timelogger = ccall.TimeHistory(save_path)
#tensorboard_callback = tf.keras.callbacks.TensorBoard("../tf_logs/"+aim_path, histogram_freq=1)


callbacks = [logger, checker, checker_best, timelogger]#, tensorboard_callback]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [23]:
model.fit(train_gen,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    #use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 160 steps, validate for 40 steps
Epoch 1/100
160/160 [==============================] - 18s 116ms/step - loss: 0.6978 - accuracy: 0.9374 - iou_score: 0.1878 - val_loss: 0.6053 - val_accuracy: 0.9624 - val_iou_score: 0.2459
Epoch 2/100
160/160 [==============================] - 7s 46ms/step - loss: 0.3578 - accuracy: 0.9888 - iou_score: 0.4752 - val_loss: 0.3387 - val_accuracy: 0.9640 - val_iou_score: 0.4940
Epoch 3/100
119/160 [=====================>........] - ETA: 1s - loss: 0.3128 - accuracy: 0.9895 - iou_score: 0.5252

KeyError: 'val_accuracy'

# Evaluation

In [24]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

In [25]:
test_gen = LASTENSequence(path_train,
                          batch_size=1,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)

# 3.2) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN2 data set.

# Data
Import training images

In [26]:
tf.keras.backend.clear_session()

path_train = r'D:\Julian\data\LASTEN2\train'
path_validation = r'D:\Julian\data\LASTEN2\validation'
path_test = r'D:\Julian\data\LASTEN2\test'

#path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/train'
#path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/validation'
#path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN2/test'

width = 768
height = 768

# Seeding
Seeds are set to ensure reproducible training.

In [27]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [28]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768, 768, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 768, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 768, 768, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 768, 768, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [29]:
train_gen = LASTENSequence(path_train,
                           batch_size=2,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=2,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

test_gen = LASTENSequence(path_train,
                          batch_size=2,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [30]:
save_path = "../results/3_2_segmentation"

logger = CSVLogger(save_path + "/log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5", period=10)
checker_best = ModelCheckpoint(save_path + "/best_weights.hdf5", save_best_only=True)
timelogger = ccall.TimeHistory(save_path)

callbacks = [logger, checker, checker_best, timelogger]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [32]:
model.fit(train_gen,
                    epochs=100,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    #use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 80 steps, validate for 20 steps
Epoch 1/100
80/80 [==============================] - 24s 296ms/step - loss: 0.7852 - accuracy: 0.9309 - iou_score: 0.1225 - val_loss: 0.9100 - val_accuracy: 0.9837 - val_iou_score: 0.0471
Epoch 2/100
29/80 [=========>....................] - ETA: 9s - loss: 0.6175 - accuracy: 0.9846 - iou_score: 0.2370

KeyError: 'val_accuracy'

# Evaluation

In [9]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

In [10]:
test_gen = LASTENSequence(path_train,
                          batch_size=1,
                          preprocess_input=pre_une,
                          width=width,
                          height=height)